In [1]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as Var 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

ImportError: numpy.core.multiarray failed to import

In [2]:
writer = SummaryWriter("my_model")

In [3]:
column_names = ["time_stamp","activity_id","heart_rate","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand",
               "imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand","imu_hand"
               ,"imu_hand","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest"
               ,"IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest","IMU_chest"
               ,"IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle"
               ,"IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle","IMU_ancle"]
subject1 = pd.read_csv(r'subject101.dat',sep = " ",names = column_names)
subject3 = pd.read_csv(r'subject104.dat',sep = " ",names = column_names)

subject1.tail()

C:\Users\hp\Anaconda3\lib\site-packages\pandas\io\parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


,time_stamp,activity_id,heart_rate,imu_hand,imu_hand.1,imu_hand.2,imu_hand.3,imu_hand.4,imu_hand.5,imu_hand.6,...,IMU_ancle.7,IMU_ancle.8,IMU_ancle.9,IMU_ancle.10,IMU_ancle.11,IMU_ancle.12,IMU_ancle.13,IMU_ancle.14,IMU_ancle.15,IMU_ancle.16
376412,3772.50,0,NaN,30.5,2.02477,7.29553,5.74194,2.06573,6.57692,5.92695,...,0.048745,-0.008034,0.018600,-58.8381,-36.2397,-11.0980,1.0,0.0,0.0,0.0
376413,3772.51,0,NaN,30.5,2.10836,7.86504,5.85674,2.08754,7.42244,5.87977,...,0.042113,0.024647,0.013375,-58.9537,-36.3379,-11.7170,1.0,0.0,0.0,0.0
376414,3772.52,0,NaN,30.5,2.07163,8.39581,5.77742,2.13833,8.05640,5.90853,...,-0.014889,0.026009,0.025054,-59.1781,-35.8985,-10.9678,1.0,0.0,0.0,0.0
376415,3772.53,0,NaN,30.5,2.19569,8.77634,6.00892,2.11251,8.53989,5.93770,...,0.047026,-0.035531,0.024640,-58.8299,-36.6845,-10.8553,1.0,0.0,0.0,0.0
376416,3772.54,0,NaN,30.5,2.14774,8.66047,5.73918,2.15810,8.59995,5.92242,...,0.042585,0.017541,-0.037894,-59.1707,-36.7731,-11.5961,1.0,0.0,0.0,0.0


In [4]:
subject1.shape

(376417, 54)

In [5]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data_full,type1, n_in=1, n_out=1, dropnan=True):
    
    data = data_full[type1].values

    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        
        agg.dropna(inplace=True)
    rest = data_full[["time_stamp","activity_id"]][84:].values
    agg.insert(0, "activity_id", rest[:,1])
    agg.insert(0, "time_stamp", rest[:,0])
       
    return agg

In [6]:
def get_series(dataset):
    
    d3 = dataset[dataset["activity_id"] == 3]
    d4 = dataset[dataset["activity_id"] == 4]
    d12 = dataset[dataset["activity_id"] == 12]
    d13 = dataset[dataset["activity_id"] == 13]
    hand = ["imu_hand.1","imu_hand.2","imu_hand.3"]
    chest = ["IMU_chest.1","IMU_chest.2","IMU_chest.3"]
    ancle = ["IMU_ancle.1","IMU_ancle.2","IMU_ancle.3"]
    imuhand_3=d3[["time_stamp","activity_id", "imu_hand.1","imu_hand.2","imu_hand.3"]].dropna()
    imuhand_3 = series_to_supervised(imuhand_3,hand,84)
    imuchest_3=d3[["time_stamp","activity_id","IMU_chest.1","IMU_chest.2","IMU_chest.3"]].dropna()
    imuchest_3 = series_to_supervised(imuchest_3,chest,84)
    imuancle_3 = d3[["time_stamp","activity_id","IMU_ancle.1","IMU_ancle.2","IMU_ancle.3"]].dropna()
    imuancle_3 = series_to_supervised(imuancle_3,ancle,84)

    imuhand_4=d4[["time_stamp","activity_id", "imu_hand.1","imu_hand.2","imu_hand.3"]].dropna()
    imuhand_4 = series_to_supervised(imuhand_4,hand,84)
    imuchest_4=d4[["time_stamp","activity_id","IMU_chest.1","IMU_chest.2","IMU_chest.3"]].dropna()
    imuchest_4 = series_to_supervised(imuchest_4,chest,84)
    imuancle_4= d4[["time_stamp","activity_id","IMU_ancle.1","IMU_ancle.2","IMU_ancle.3"]].dropna()
    imuancle_4 = series_to_supervised(imuancle_4,ancle,84)

    imuhand_12=d12[["time_stamp","activity_id", "imu_hand.1","imu_hand.2","imu_hand.3"]].dropna()
    imuhand_12 = series_to_supervised(imuhand_12,hand,84)
    imuchest_12=d12[["time_stamp","activity_id","IMU_chest.1","IMU_chest.2","IMU_chest.3"]].dropna()
    imuchest_12 = series_to_supervised(imuchest_12,chest,84)
    imuancle_12 = d12[["time_stamp","activity_id","IMU_ancle.1","IMU_ancle.2","IMU_ancle.3"]].dropna()
    imuancle_12 = series_to_supervised(imuancle_12,ancle,84)

    imuhand_13=d13[["time_stamp","activity_id", "imu_hand.1","imu_hand.2","imu_hand.3"]].dropna()
    imuhand_13 = series_to_supervised(imuhand_13,hand,84)
    imuchest_13=d13[["time_stamp","activity_id","IMU_chest.1","IMU_chest.2","IMU_chest.3"]].dropna()
    imuchest_13 = series_to_supervised(imuchest_13,chest,84)
    imuancle_13= d13[["time_stamp","activity_id","IMU_ancle.1","IMU_ancle.2","IMU_ancle.3"]].dropna()
    imuancle_13 = series_to_supervised(imuancle_13,ancle,84)

    merged_hand = pd.concat([imuhand_3,imuhand_4,imuhand_12,imuhand_13],ignore_index = True)
    merged_chest = pd.concat([imuchest_3,imuchest_4,imuchest_12,imuchest_13],ignore_index = True)
    merged_ancle = pd.concat([imuancle_3,imuancle_4,imuancle_12,imuancle_13],ignore_index = True)

    return merged_hand,merged_chest, merged_ancle

In [7]:
sbj_1_hand, sbj_1_chest, sbj_1_ancle = get_series(subject1)
print(sbj_1_hand.shape)
print(sbj_1_chest.shape)
print(sbj_1_ancle.shape)

(73725, 257)
(74237, 257)
(74271, 257)


In [8]:
sbj_3_hand, sbj_3_chest, sbj_3_ancle = get_series(subject3)
print(sbj_3_hand.shape)
print(sbj_3_chest.shape)
print(sbj_3_ancle.shape)

(85738, 257)
(87212, 257)
(86877, 257)


In [9]:
from sklearn.utils import shuffle
sbj_3_hand = shuffle(sbj_3_hand, random_state=0)
sbj_3_chest = shuffle(sbj_3_chest, random_state=0)
sbj_3_ancle = shuffle(sbj_3_ancle, random_state=0)
sbj_1_hand = shuffle(sbj_1_hand, random_state=0)
sbj_1_chest = shuffle(sbj_1_chest, random_state=0)
sbj_1_ancle = shuffle(sbj_1_ancle, random_state=0)

In [12]:
dataset_hand = sbj_1_hand

dataset_chest = sbj_1_chest[:73725]

dataset_ancle =sbj_1_ancle[:73725]

In [13]:
print(len(dataset_hand))
print(len(dataset_chest))
print(len(dataset_ancle))

73725
73725
73725


In [14]:
def data_gen(data):
    target = data[["activity_id"]]
    data.pop("activity_id")
    return data,target

In [15]:
x_chest,y_chest = data_gen(dataset_chest)
x_hand, y_hand = data_gen(dataset_hand)
x_ancle,y_ancle = data_gen(dataset_ancle)
x_test_hand,y_test_hand = data_gen(sbj_3_hand)
x_test_chest,y_test_chest = data_gen(sbj_3_chest)
x_test_ancle,y_test_ancle = data_gen(sbj_3_ancle)

In [16]:
x_test_chest.shape

(87212, 256)

In [17]:
def norm(data, norm_data):
    train_stats = data.describe()
    train_stats = train_stats.transpose()
    return(norm_data - train_stats['mean'])/ train_stats['std']

In [20]:

x_hand = x_hand.values.reshape(73725,1,256)


x_chest = x_chest.values.reshape(73725,1,256)


x_ancle = x_ancle.values.reshape(73725,1,256)

In [21]:
xtest_hand = x_test_hand[:1000]
test_hand = xtest_hand.values.reshape(1000,1,256)
xtest_chest = x_test_chest[:1000]
test_chest = xtest_chest.values.reshape(1000,1,256)
xtest_ancle = x_test_ancle[:1000]
test_ancle = xtest_ancle.values.reshape(1000,1,256)

In [22]:
def recode_target(data):
    cont = []
    for i in data["activity_id"]:
        if i == 3:
            cont.append(0)
        elif i == 4:
            cont.append(1)
        elif i == 12:
            cont.append(2)
        elif i == 13:
            cont.append(3)
    data["activity_id"] = cont
    
    return data
            
    

In [23]:

y_train1 =y_hand
y_train = recode_target(y_train1)

y_test1 = y_test_hand[:1000]
y_test = recode_target(y_test1)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [24]:
head = data_utils.TensorDataset(torch.tensor(x_hand), torch.tensor(y_train.values))
train_loader1 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)
head = data_utils.TensorDataset(torch.tensor(x_chest), torch.tensor(y_train.values))
train_loader2 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)
head = data_utils.TensorDataset(torch.tensor(x_ancle), torch.tensor(y_train.values))
train_loader3 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)

In [25]:
head = data_utils.TensorDataset(torch.tensor(test_hand), torch.tensor(y_test.values))
train_loader_test1 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)
head = data_utils.TensorDataset(torch.tensor(test_chest), torch.tensor(y_test.values))
train_loader_test2 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)
head = data_utils.TensorDataset(torch.tensor(test_ancle), torch.tensor(y_test.values))
train_loader_test3 = data_utils.DataLoader(head, batch_size=1000, shuffle=False)

In [26]:
for i,(z,j,q) in enumerate(zip(train_loader1,train_loader2,train_loader3)):
    f= i
    (d,g) = z
    (o,u) = j
    (s,a) = q
print(d.shape)
print(o)
print(s)
print(f)

torch.Size([725, 1, 256])
tensor([[[ 5.9501e+02,  6.3400e-01,  9.7646e+00,  ...,  7.9779e-01,
           9.7290e+00, -7.8801e-01]],

        [[ 1.5885e+03,  1.8174e+00,  7.4019e+00,  ...,  4.4860e-01,
           5.0914e+00, -1.8635e+00]],

        [[ 1.8695e+03,  6.3769e-02,  4.5649e+00,  ...,  1.3717e+00,
           1.4990e+01, -7.6483e+00]],

        ...,

        [[ 6.6922e+02,  3.5337e-01,  9.7246e+00,  ...,  3.2107e-01,
           9.7637e+00, -1.4550e+00]],

        [[ 1.9832e+03, -8.4136e-01,  1.0365e+01,  ...,  5.7770e-01,
           3.3257e+00, -1.1971e+00]],

        [[ 2.3083e+03,  1.1770e-01,  6.2241e+00,  ...,  5.3346e-01,
           1.3108e+01, -2.3139e+00]]], dtype=torch.float64)
tensor([[[ 1.9372e+03,  9.7627e+00, -1.6538e+00,  ...,  9.7266e+00,
          -1.3878e+00, -4.0948e-01]],

        [[ 2.3825e+03,  2.2762e+00, -2.6981e+00,  ...,  1.0969e+01,
           2.1005e-01, -1.7573e+00]],

        [[ 6.2865e+02,  9.3503e+00, -2.6290e+00,  ...,  9.4505e+00,
          -2.81

In [27]:
class CNN_Model(nn.Module):
    def __init__(self):
        
        super(CNN_Model, self).__init__()
        
        self.conv11 = nn.Conv1d(in_channels = 1, out_channels = 8, kernel_size = 5, padding=0)
        self.pool11 = nn.AvgPool1d(kernel_size=2)
        self.dropout = nn.Dropout(p = 0.5)
        self.conv12 = nn.Conv1d(in_channels = 8, out_channels = 4, kernel_size = 5,padding = 0)
        self.pool12 = nn.AvgPool1d(kernel_size = 2)
        
        self.conv21 = nn.Conv1d(in_channels = 1, out_channels = 8, kernel_size = 5, padding=0)
        self.pool21 = nn.AvgPool1d(2)
        self.dropout = nn.Dropout(p = 0.5)
        self.conv22 = nn.Conv1d(in_channels = 8, out_channels = 4, kernel_size = 5,padding = 0)
        self.pool22 = nn.AvgPool1d(2)
        
        self.conv31 = nn.Conv1d(in_channels = 1, out_channels = 8, kernel_size = 5, padding=0)
        self.pool31 = nn.AvgPool1d(2)
        self.dropout = nn.Dropout(p = 0.5)
        self.conv32 = nn.Conv1d(in_channels = 8, out_channels = 4, kernel_size = 5,padding = 0)
        self.pool32 = nn.AvgPool1d(2)
        
        
        self.fc1 = nn.Linear(732, 256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64, 4)
        
    def forward(self,x):
        
        in_size1 = len(x[0])
        x1 = self.conv11(x[0])
        x1 = self.pool11(x1)
        x1 = F.relu(x1)
        x1 = self.dropout(x1)
        x1 = F.relu(self.pool12(self.conv12(x1)))
        x1 = x1.view(in_size1, -1)
        
        in_size2 = len(x[1])
        x2 = F.relu(self.pool21(self.conv21(x[0])))
        x2 = self.dropout(x2)
        x2 = F.relu(self.pool22(self.conv22(x2)))
        x2 = x2.view(in_size2, -1)
        
        in_size3 = len(x[2])
        x3 = F.relu(self.pool31(self.conv31(x[0])))
        x3 = self.dropout(x3)
        x3 = F.relu(self.pool32(self.conv32(x3)))
        x3 = x3.view(in_size3, -1)
        
        x = torch.cat((x1,x2,x3),1)
        #print(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #print(x)
        x = F.softmax(self.fc3(x))
        
        return x
        
        

In [28]:

model = CNN_Model()


optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.5)

In [29]:
def train(epoch):
    model1 = model.double()
    model1.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data1,data2,data3) in enumerate(zip(train_loader1,train_loader2,train_loader3)):
        (head_data,target) = data1
        (chest_data,target) = data2
        (ancle_data,target) = data3
        target = target
        head_data, target = Var(head_data), Var(target.long())
        
        chest_data, target = Var(chest_data), Var(target.long())
        
        ancle_data, target1 = Var(ancle_data), Var(target.long())
        
        optimizer.zero_grad()
        output = model1(([head_data,chest_data,ancle_data]))
        
        target1 = target1.squeeze()
        loss = F.nll_loss(output, target1)
        loss.backward()
        
        optimizer.step()
        train_loss += F.nll_loss(output, target1, size_average=False).data
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target1.data.view_as(pred)).cpu().sum()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data1), len(train_loader1.dataset),
                100. * batch_idx / len(train_loader1), loss.item()))
    
    
    
    train_loss /= len(train_loader_test1.dataset)
    writer.add_scalar("Loss/train_loss", train_loss,epoch)
    print('\nTrain_Accuracy: {}/{} ({:.0f}%)\n'.format(
         correct, len(train_loader1.dataset),
        100. * correct / len(train_loader1.dataset)))
    writer.add_scalar("Accuracy/train_accuracy", 100. * correct / len(train_loader_test1.dataset),epoch)

In [30]:
def test(epoch):
    model1 = model.double()
    model1.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data1,data2,data3) in enumerate(zip(train_loader_test1,train_loader_test2,train_loader_test3)):
        (head_data,target) = data1
        (chest_data,target) = data2
        (ancle_data,target) = data3
        target = target
        head_data, target = Var(head_data ), Var(target.long())
        
        chest_data, target = Var(chest_data ), Var(target.long())
        
        ancle_data, target1 = Var(ancle_data), Var(target.long())
        
        output = model1(([head_data,chest_data,ancle_data]))
        
        target1 = target1.squeeze()
        test_loss += F.nll_loss(output, target1, size_average=False).data
        
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target1.data.view_as(pred)).cpu().sum()

    test_loss /= len(train_loader_test1.dataset)
    writer.add_scalar("Loss/test_loss",test_loss,epoch)
    print('\nTest set: Average loss: {:.4f}, Test_Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(train_loader_test1.dataset),
        100. * correct / len(train_loader_test1.dataset)))
    writer.add_scalar("Accuracy/test_accuracy", 100. * correct / len(train_loader_test1.dataset),epoch)
    

In [31]:
for epoch in range(10):
    train(epoch)
    test(epoch)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\hp\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 0 [0/73725 (0%)]	Loss: -0.243968
Train Epoch: 0 [20/73725 (14%)]	Loss: -0.250002
Train Epoch: 0 [40/73725 (27%)]	Loss: -0.263517
Train Epoch: 0 [60/73725 (41%)]	Loss: -0.287951
Train Epoch: 0 [80/73725 (54%)]	Loss: -0.307714
Train Epoch: 0 [100/73725 (68%)]	Loss: -0.310898
Train Epoch: 0 [120/73725 (81%)]	Loss: -0.323094
Train Epoch: 0 [140/73725 (95%)]	Loss: -0.330241

Train_Accuracy: 20006/73725 (27%)


Test set: Average loss: -0.3864, Test_Accuracy: 360/1000 (36%)

Train Epoch: 1 [0/73725 (0%)]	Loss: -0.334963
Train Epoch: 1 [20/73725 (14%)]	Loss: -0.322531
Train Epoch: 1 [40/73725 (27%)]	Loss: -0.319320
Train Epoch: 1 [60/73725 (41%)]	Loss: -0.330237
Train Epoch: 1 [80/73725 (54%)]	Loss: -0.329980
Train Epoch: 1 [100/73725 (68%)]	Loss: -0.329505
Train Epoch: 1 [120/73725 (81%)]	Loss: -0.336415
Train Epoch: 1 [140/73725 (95%)]	Loss: -0.342190

Train_Accuracy: 21923/73725 (30%)


Test set: Average loss: -0.4017, Test_Accuracy: 360/1000 (36%)

Train Epoch: 2 [0/73725 (0%)